In [ ]:
import pandas as pd
import datetime as dt
#from datetime import datetime, timedelta
import numpy as np
import os
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)



import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.impute import SimpleImputer



# Bayesian optimization
import warnings
warnings.filterwarnings("ignore")
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import KFold, cross_val_score


In [ ]:
data=pd.read_pickle("full_dataframe_20210803_150443.pkl")
data.reset_index(inplace=True)
data.drop(['index','event_id'], inplace=True, axis=1)
print(data.shape)
data.head()

In [ ]:
data=data.head(500)

In [ ]:
data.shape

In [ ]:
train, test = train_test_split(data, test_size=0.20, random_state=42)

In [ ]:
Y_train = train["COLLISSION_PROBABILITY_TARGET"]
X_train= train.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)
Y_test = test["COLLISSION_PROBABILITY_TARGET"]
X_test= test.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)


In [ ]:
X = X_train
y = Y_train

In [ ]:
""" params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
train_data = lgb.Dataset(data=X, label=y, categorical_feature = list(X.columns),free_raw_data=False)
    
cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False,shuffle=False) """



In [ ]:
""" def lgb_eval(num_leaves, feature_fraction, max_depth , min_split_gain, min_child_weight):
    params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['max_depth'] = int(round(max_depth))
    params['num_leaves'] = int(round(num_leaves))
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False)
    return (-1.0 * np.array(cv_result['rmse-mean'])).max() """

In [ ]:
""" lgbBO = BayesianOptimization(lgb_eval, {'feature_fraction': (0.1, 0.9),
                                            'max_depth': (5, 9),
                                            'num_leaves' : (200,300),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0) """


In [ ]:
#lgbBO.maximize(init_points=5, n_iter=5,acq='ei')

In [ ]:
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=0, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, categorical_feature = list(X.columns),free_raw_data=False)
    # parameters

    def lgb_eval(num_leaves, feature_fraction, max_depth , min_split_gain, min_child_weight):
        params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['num_leaves'] = int(round(num_leaves))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, verbose_eval =200,stratified=False)
        return (-1.0 * np.array(cv_result['rmse-mean'])).max()
    
        # range 
    lgbBO = BayesianOptimization(lgb_eval, {'feature_fraction': (0.1, 0.9),
                                            'max_depth': (5, 9),
                                            'num_leaves' : (200,300),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
        # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round,acq='ei')

        # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")

        # return best parameters
    return lgbBO.res['max']['max_params']

opt_params = bayes_parameter_opt_lgb(X, y, init_round=10, opt_round=10, n_folds=5, random_seed=0, n_estimators=1000, learning_rate=0.01)